# Backtesting with backtrader library
Notes:  
Timestamp must be in the exact format specified for datafeed   **Timestamp needs to be first column index**  
Params for datafeed need to be specified as tuple of tuples  
Within strategy data is accessed via self.datas[0] for ex: self.datas[0].high_delta[0] would get you the first timeperiod's high_delta  
The current data has already happened and cannot be used to execute an order (ex. you cannot look at the current close price and simultaneously buy the current close, the order will be executed at open instead), orders will be executed on the following day.

In [30]:
import backtrader as bt
import datetime
import pandas as pd

## Define Strategies and Data utils

In [31]:
class BasicStrategy(bt.Strategy):
    def __init__(self):
        # To keep track of pending orders
        self.order = None
        print('***The limit sells for this strategy are valid for 1 day***')
        
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

            # Note: This logic does not properly work
#         elif order.status in [order.Expired]:
#             if order.isbuy():
#                 self.log('BUY ORDER EXPIRED')
#             elif order.issell():
#                 self.log('SELL ORDER EXPIRED')
        
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None



    def next(self):
        # STRATEGY 
        data = self.datas[0]
        self.log(f'Current Portfolio Value : {self.broker.get_value()}')
        
        # Check if we are in the market
        if not self.position:
            # BUY
            try:
                self.size = int(self.broker.get_cash() / self.datas[0].open[1])
            except:
                print('Size Exception. If at the end of data, ignore.')
            # invest if prediction looks good
            # TODO: Make this a filter in the class constructor
            if (data.prediction[0] >= data.close[0]):
                self.log(f'MARKET BUY CREATE {self.size} shares at next open, current close price: {data.close[0]}')
                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy(size=self.size) # market order buys at next open
                
        else:
            # place sell order at predicted high if predicted high is greater than current close price
            if data.prediction[0] > data.close[0]:
                self.log(f'LIMIT SELL CREATE {self.size} shares at {data.prediction[0]}')
                # Keep track of the created order to avoid a 2nd order, order valid for 1 day
                self.order = self.sell(exectype=bt.Order.Limit,
                             price=data.prediction[0],
                             valid=data.datetime.date(0) + datetime.timedelta(days=2),
                             size=self.size)
            # if prediction is less than current value sell at open
            else:
                self.log('MARKET SELL CREATE. PREDICTION < CURRENT CLOSE')
                self.order = self.sell(size=self.size)
        

In [32]:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

      
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.datas[0].open[1])
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print(f'Stop price: {self.datas[0].close[0]}')
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [33]:
def prepare_data(data, fromdate, todate, filepath):
    """Prepare data for backtrader datafeed object
        Returns prepared data filepath and params for the GenericCSVData class, also returns columns used"""
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    # start param setup for backtrader
    start = data['timestamp'].iloc[0]
    end = data['timestamp'].iloc[-1]
    from_to = [(start.year, start.month, start.day), (end.year, end.month, end.day)]
    # Backtrader string format
    data['timestamp'] = data['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
    # backtrader data feed class needs a file path, this could be a temp file that's constantly overwritten
    data.to_csv(filepath)
    
    starting_params = [
    ('fromdate', datetime.datetime(*from_to[0])),
    ('todate', datetime.datetime(*from_to[1])),
    ('nullvalue', 0.0),
    ('dtformat', ('%Y-%m-%d %H:%M:%S')),
    ('tmformat', ('%H:%M:%S')),
    ('datetime', 1)]
    # skip nonfeatures (timestamp)
    cols = data.columns[1:]
    # get column position for each indicator and add to starting params list
    i = 2 # starting index since others are reserved
    for indicator in cols:
        starting_params.append((indicator, i))
        i+=1
    final_params = tuple(starting_params)

    return filepath, final_params, cols 


## Import Data and Run Backtest

In [34]:
import os

In [35]:
def capstone_data_prep(pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000):
    """Main loop, preps data and executes backtrader on all stocks in the prediction folder"""
    ticker_backtesting_dict = {}
    for pred_file in os.listdir(f'../data/ticker_predictions/{pred_folder_name}'):
        ticker_name = pred_file.split('_')[0]
        pred_df = pd.read_csv(f'../data/ticker_predictions/{pred_folder_name}/{pred_file}')
        ticker_df = pd.read_csv(ticker_df_filepath + f'/{ticker_name}_full_data.csv')
        # new prediction column name = "prediction", important because backtester strategy looks for this column name
        ticker_df['prediction'] = pred_df[pred_col_name]
        # remove all non-prediction rows, use "timestamp" as the common datetime name
        ticker_df = ticker_df[ticker_df.prediction > 0]
        ticker_df = ticker_df.rename({'reportperiod':'timestamp'}, axis=1)

        # make timestamp in column position 0 
        ticker_cols = list(ticker_df.columns)
        ticker_cols[5] = 'ts_cpy'
        ticker_cols[0] = 'timestamp'
        ticker_df.columns = ticker_cols
        ticker_df['timestamp'] = ticker_df['ts_cpy']

        # skip nonfeatures, keep timestamp
        nonfeatures = ticker_df.columns[1:7]
        features = [col for col in ticker_df.columns if col not in nonfeatures]
        # get index of start and end dates for trading
        ticker_df = ticker_df[features]
        ticker_df.columns = [c.lower() for c in ticker_df.columns]
        idx1 = ticker_df.prediction[ticker_df.prediction > 0].index[0]
        idx2 = ticker_df.prediction[ticker_df.prediction > 0].index[-1]
        start_date = pred_df.timestamp[idx1]
        end_date  = pred_df.timestamp[idx2]

        # Prepare data for backtrader
        prep_data = prepare_data(ticker_df.copy(), start_date, end_date, 'temp/prep_data.csv')
        class DataFeed(bt.feeds.GenericCSVData):
            lines = tuple(prep_data[2])
            params = prep_data[1]
        
        print(f"--------Ticker Name-------: {ticker_name}")
        ##### BUY AND HOLD #######
        print('BUY AND HOLD')
        # initialize everything and run strategy
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        #cerebro.addsizer(bt.sizers.FixedSize, stake=10)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BuyAndHold)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        buy_hold = cerebro.run()
        buy_hold_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(buy_hold[0].analyzers.sharpe.get_analysis())
        print(buy_hold[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(buy_hold[0].analyzers.returns.get_analysis())

        #### MAIN STRATEGY #####
        cerebro = bt.Cerebro(cheat_on_open=True)
        cerebro.broker.setcash(starting_cash)
        print(f'Starting Portfolio Value: {cerebro.broker.getvalue()}')
        data = DataFeed(dataname=prep_data[0])
        cerebro.adddata(data)
        cerebro.addstrategy(BasicStrategy)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Months, _name = 'sharpe')
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name = 'dd')
        cerebro.addanalyzer(bt.analyzers.Returns, _name = 'returns')
        back = cerebro.run()
        strat_ret = cerebro.broker.getvalue() / starting_cash - 1
        print(f'Final Portfolio Value: {cerebro.broker.getvalue()}\n')
        print('STRATEGY INFO:')
        print(back[0].analyzers.sharpe.get_analysis())
        print(back[0].analyzers.dd.get_analysis())
        # rtot is total log returns over the strategy time period
        print(back[0].analyzers.returns.get_analysis())
        print(f"For {ticker_name}, return was {buy_hold_ret*100}% for buy and hold vs {strat_ret*100}%  for strategy")
        print('\n'*5)

        ticker_backtesting_dict[ticker_name] = cerebro
        
    return ticker_backtesting_dict
#     return ticker_df

In [36]:
# pred_folder_name, pred_col_name, ticker_df_filepath, starting_cash=100000

cerebro_dict = capstone_data_prep('ar_preds', 'prediction', '../data/ticker_data')
cerebro_dict

--------Ticker Name-------: MSFT
BUY AND HOLD
Starting Portfolio Value: 100000
2000-04-03, BUY EXECUTED, 47.22
Stop price: 25.45499992370605
ROI:        -46.07%
Final Portfolio Value: 53926.14108848571

STRATEGY INFO:
OrderedDict([('sharperatio', -0.08149073316471005)])
AutoOrderedDict([('len', 541), ('drawdown', 46.07385891151429), ('moneydown', 46073.85891151429), ('max', AutoOrderedDict([('len', 541), ('drawdown', 56.03434375), ('moneydown', 56034.34375)]))])
OrderedDict([('rtot', -0.6175548332786981), ('ravg', -0.001139400061399812), ('rnorm', -0.24958494245957547), ('rnorm100', -24.958494245957546)])
Starting Portfolio Value: 100000
***The limit sells for this strategy are valid for 1 day***
2000-03-31, Current Portfolio Value : 100000.0
2000-03-31, MARKET BUY CREATE 2117 shares at next open, current close price: 53.125
2000-04-03, BUY EXECUTED, 47.22
2000-04-03, Current Portfolio Value : 96229.09375
2000-04-03, LIMIT SELL CREATE 2117 shares at 139.52539573803514
2000-04-04, Curre

2000-08-17, Current Portfolio Value : 75786.8125
2000-08-17, LIMIT SELL CREATE 2117 shares at 181.42673053598173
2000-08-18, Current Portfolio Value : 75191.40625
2000-08-18, LIMIT SELL CREATE 2117 shares at 181.20513324935
2000-08-21, Current Portfolio Value : 74794.46875
2000-08-21, LIMIT SELL CREATE 2117 shares at 184.96220473756827
2000-08-22, Current Portfolio Value : 75456.03125
2000-08-22, LIMIT SELL CREATE 2117 shares at 185.22408545382007
2000-08-23, Current Portfolio Value : 74926.78125
2000-08-23, LIMIT SELL CREATE 2117 shares at 190.0689786061542
2000-08-24, Current Portfolio Value : 75323.71875
2000-08-24, LIMIT SELL CREATE 2117 shares at 185.77807867039945
2000-08-25, Current Portfolio Value : 74794.46875
2000-08-25, LIMIT SELL CREATE 2117 shares at 186.05004177117272
2000-08-28, Current Portfolio Value : 75522.1875
2000-08-28, LIMIT SELL CREATE 2117 shares at 185.04278696629697
2000-08-29, Current Portfolio Value : 75125.25
2000-08-29, LIMIT SELL CREATE 2117 shares at 18

2000-12-27, Current Portfolio Value : 49192.0
2000-12-27, LIMIT SELL CREATE 2117 shares at 190.3107252923403
2000-12-28, Current Portfolio Value : 47207.3125
2000-12-28, LIMIT SELL CREATE 2117 shares at 194.9743199571513
2000-12-29, Current Portfolio Value : 45950.34375
2000-12-29, LIMIT SELL CREATE 2117 shares at 195.64917883207943
2001-01-02, Current Portfolio Value : 45950.34375
2001-01-02, LIMIT SELL CREATE 2117 shares at 197.7442706705596
2001-01-03, Current Portfolio Value : 50779.75
2001-01-03, LIMIT SELL CREATE 2117 shares at 196.56576902370503
2001-01-04, Current Portfolio Value : 51309.0
2001-01-04, LIMIT SELL CREATE 2117 shares at 202.02510359128144
2001-01-05, Current Portfolio Value : 52036.71875
2001-01-05, LIMIT SELL CREATE 2117 shares at 203.37482134113765
2001-01-08, Current Portfolio Value : 51838.25
2001-01-08, LIMIT SELL CREATE 2117 shares at 199.2752869079389
2001-01-09, Current Portfolio Value : 54881.4375
2001-01-09, LIMIT SELL CREATE 2117 shares at 201.793423920

2001-06-11, Current Portfolio Value : 76376.92915725708
2001-06-11, LIMIT SELL CREATE 2117 shares at 211.6343076672211
2001-06-12, Current Portfolio Value : 76334.58818817139
2001-06-12, LIMIT SELL CREATE 2117 shares at 215.41151318550507
2001-06-13, Current Portfolio Value : 74863.27383422852
2001-06-13, LIMIT SELL CREATE 2117 shares at 215.4215802005379
2001-06-14, Current Portfolio Value : 72968.55786514282
2001-06-14, LIMIT SELL CREATE 2117 shares at 216.79144734994844
2001-06-15, Current Portfolio Value : 72037.07269668579
2001-06-15, LIMIT SELL CREATE 2117 shares at 215.62304345710396
2001-06-18, Current Portfolio Value : 70830.38334274292
2001-06-18, LIMIT SELL CREATE 2117 shares at 217.7785681247585
2001-06-19, Current Portfolio Value : 71296.12592697144
2001-06-19, LIMIT SELL CREATE 2117 shares at 216.93246240785152
2001-06-20, Current Portfolio Value : 73508.39512634277
2001-06-20, LIMIT SELL CREATE 2117 shares at 215.7942749295942
2001-06-21, Current Portfolio Value : 73963.

2001-11-29, Current Portfolio Value : 68671.04237365723
2001-11-29, LIMIT SELL CREATE 2117 shares at 240.7338983653016
2001-11-30, Current Portfolio Value : 68004.1902809143
2001-11-30, LIMIT SELL CREATE 2117 shares at 244.79313399939173
2001-12-03, Current Portfolio Value : 68596.94769668579
2001-12-03, LIMIT SELL CREATE 2117 shares at 249.9502736821192
2001-12-04, Current Portfolio Value : 69898.90625
2001-12-04, LIMIT SELL CREATE 2117 shares at 246.95872568207912
2001-12-05, Current Portfolio Value : 72121.75463485718
2001-12-05, LIMIT SELL CREATE 2117 shares at 244.8434998135332
2001-12-06, Current Portfolio Value : 72703.93286514282
2001-12-06, LIMIT SELL CREATE 2117 shares at 244.8837832431533
2001-12-07, Current Portfolio Value : 71835.96318817139
2001-12-07, LIMIT SELL CREATE 2117 shares at 248.26816000231528
2001-12-10, Current Portfolio Value : 71020.91366577148
2001-12-10, LIMIT SELL CREATE 2117 shares at 246.94865866704623
2001-12-11, Current Portfolio Value : 71296.1259269

...

2000-09-12, Current Portfolio Value : 153997.27066135406
2000-09-12, LIMIT SELL CREATE 13473 shares at 256.7341907612203
2000-09-13, Current Portfolio Value : 154102.515625
2000-09-13, LIMIT SELL CREATE 13473 shares at 275.010469217361
2000-09-14, Current Portfolio Value : 159154.890625
2000-09-14, LIMIT SELL CREATE 13473 shares at 263.21899172726893
2000-09-15, Current Portfolio Value : 156102.42691135406
2000-09-15, LIMIT SELL CREATE 13473 shares at 291.4339934923979
2000-09-18, Current Portfolio Value : 153576.23941135406
2000-09-18, LIMIT SELL CREATE 13473 shares at 284.9290183207001
2000-09-19, Current Portfolio Value : 158839.13003635406
2000-09-19, LIMIT SELL CREATE 13473 shares at 285.8453544033375
2000-09-20, Current Portfolio Value : 155470.88003635406
2000-09-20, LIMIT SELL CREATE 13473 shares at 275.3427827891656
2000-09-21, Current Portfolio Value : 159049.64566135406
2000-09-21, LIMIT SELL CREATE 13473 shares at 281.94842924449864
2000-09-22, Current Portfolio Value : 164

2001-03-05, Current Portfolio Value : 202100.078125
2001-03-05, LIMIT SELL CREATE 13473 shares at 319.0145756449962
2001-03-06, Current Portfolio Value : 203817.8799715042
2001-03-06, LIMIT SELL CREATE 13473 shares at 317.59476022442226
2001-03-07, Current Portfolio Value : 203784.203125
2001-03-07, LIMIT SELL CREATE 13473 shares at 315.5103483901791
2001-03-08, Current Portfolio Value : 207152.453125
2001-03-08, LIMIT SELL CREATE 13473 shares at 316.32601318918125
2001-03-09, Current Portfolio Value : 205434.6512784958
2001-03-09, LIMIT SELL CREATE 13473 shares at 310.9891137690892
2001-03-12, Current Portfolio Value : 200954.87261104584
2001-03-12, LIMIT SELL CREATE 13473 shares at 314.39263901033104
2001-03-13, Current Portfolio Value : 198395.00826454166
2001-03-13, LIMIT SELL CREATE 13473 shares at 310.96898565835016
2001-03-14, Current Portfolio Value : 195700.39798545834
2001-03-14, LIMIT SELL CREATE 13473 shares at 313.8388240305768
2001-03-15, Current Portfolio Value : 195060.

2001-08-17, Current Portfolio Value : 221029.65237617493
2001-08-17, LIMIT SELL CREATE 13473 shares at 340.24128174117
2001-08-20, Current Portfolio Value : 222612.7082901001
2001-08-20, LIMIT SELL CREATE 13473 shares at 331.61163851375096
2001-08-21, Current Portfolio Value : 224802.07592964172
2001-08-21, LIMIT SELL CREATE 13473 shares at 326.768169741196
2001-08-22, Current Portfolio Value : 226957.75387382507
2001-08-22, LIMIT SELL CREATE 13473 shares at 326.59697324512393
2001-08-23, Current Portfolio Value : 231572.265625
2001-08-23, LIMIT SELL CREATE 13473 shares at 331.92379324487644
2001-08-24, Current Portfolio Value : 229416.58768081665
2001-08-24, LIMIT SELL CREATE 13473 shares at 335.72001639014
2001-08-27, Current Portfolio Value : 230730.203125
2001-08-27, LIMIT SELL CREATE 13473 shares at 335.3877335482755
2001-08-28, Current Portfolio Value : 232178.55165290833
2001-08-28, LIMIT SELL CREATE 13473 shares at 330.52410593504146
2001-08-29, Current Portfolio Value : 233256

2002-01-10, Current Portfolio Value : 237803.52298545837
2002-01-10, LIMIT SELL CREATE 13473 shares at 401.4643569422851
2002-01-11, Current Portfolio Value : 241677.015625
2002-01-11, LIMIT SELL CREATE 13473 shares at 398.31257137001154
2002-01-14, Current Portfolio Value : 244203.203125
2002-01-14, LIMIT SELL CREATE 13473 shares at 401.6456021287567
2002-01-15, Current Portfolio Value : 244708.44576454163
2002-01-15, LIMIT SELL CREATE 13473 shares at 407.7679280868322
2002-01-16, Current Portfolio Value : 245887.328125
2002-01-16, LIMIT SELL CREATE 13473 shares at 404.25362141167545
2002-01-17, Current Portfolio Value : 243664.2707901001
2002-01-17, LIMIT SELL CREATE 13473 shares at 400.8702550111127
2002-01-18, Current Portfolio Value : 244472.66929244995
2002-01-18, LIMIT SELL CREATE 13473 shares at 403.22653462506327
2002-01-22, Current Portfolio Value : 244371.62590408325
2002-01-22, LIMIT SELL CREATE 13473 shares at 407.707513024675
2002-01-23, Current Portfolio Value : 243866.3

{'MSFT': <backtrader.cerebro.Cerebro at 0x7fb660776410>,
 'ADSK': <backtrader.cerebro.Cerebro at 0x7fb6609fea10>,
 'XOM': <backtrader.cerebro.Cerebro at 0x7fb690825c50>,
 'WAT': <backtrader.cerebro.Cerebro at 0x7fb6904be050>,
 'HD': <backtrader.cerebro.Cerebro at 0x7fb67044e350>,
 'UNH': <backtrader.cerebro.Cerebro at 0x7fb680f877d0>}

### Library additionally has plotting
Not sure how to reformat this, it's pretty ugly

In [22]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams.update({'font.size': 12}) 
cerebro_dict['UNH'].plot()

ImportError: cannot import name 'warnings' from 'matplotlib.dates' (/opt/anaconda3/lib/python3.7/site-packages/matplotlib/dates.py)

### Ways to Improve:

Increase strategy trading volume. There are certain periods where sell orders are not going through for multiple days in a row, meaning that the price is lower than anticipated by model for that period. For example, if the last sell order didn't go through, we can find a way to use that information to inform our next sell order and make the prediction lower or try a market sell. Maybe train a "bear" model, one that only knows pain and suffering of bear markets, and only produces pessimistic high price predictions.   

Smarter entry points. Currently, buy orders are being executed at market price at the open. Returns may increase if entry constraints are increased.  

Resizing orders. Orders for this strategy baseline are currently 100% in or 100% out of position. This has beneifits if you want less exposure to the long term movement of the stocks. Depending on the situation and strategy, we might want to reserve a cash position to expose ourselves less to short term movements.  